# Lunatic to params files for mass sample concentration adjustment on TECAN

This example notebook demonstrates how to use report generated by an UNchained Labs Lunatic instrument and convert the concentration readings to a params.txt file that can be imported into TECAN Fluent to do mass concentration adjustments.

When you run the codes, you get to specify:<br>
The **folder** where your lunatic report is stored (must be in the same folder as this .ipynb file).<br>
The **file name** of your lunatic report.<br>
The **final volume** of the sample after adjustment.<br>
The **final concentrations** of the sample after adjustment.<br>
<br>
The ouptput files will be stored in an output folder you specified. If no output folder exist before hand, a new output folder will be created. <br>

*Note: <br>
1. If **starting concentration** is below **final concentration**, no dilution occurs and final concentration will be the same as starting concentration. TECAN will pipet a full final volume of the sample with starting concentration in to the target plate. <br>
2. If sample concentration is 0 then no sample or buffer will be added to the corresponding well. 

# Import all the packages and define the specifics:

In [1]:
# import packages
import pandas as pd
import os

print("Please specify the folder name where you stored your .xlsx report. Must be in the same folder as this ipynb file.")
folder_name = input()

print("These are the files in the folder, which one would you like to process?")
! ls "{folder_name}"

raw_data = input()
print("\nThis is the file to be processed:", raw_data)

print("\nPlease specify the final volume below, unit: uL")
final_volume = input()                
final_volume = int(final_volume)

print("Please specify the final concentration below, unit: mg/ml")
final_concentration = input()
final_concentration = float(final_concentration)

print("Please specify the output folder name you would like to store the output files. \nPlease do not include spaces in your folder name.")
# Create the output folder if it doesn't exist
output_folder = input()
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print("\nThis is the output folder your params files will be stored:", output_folder)


Please specify the folder name where you stored your .xlsx report. Must be in the same folder as this ipynb file.


 params_raw


These are the files in the folder, which one would you like to process?
2024-04-03_105044_401285_DPD__H10ss_TECAN_POROS_Screen.xlsx
2025-02-28_110706_401285_DPD__Mumps_verification_run_repeat.xlsx
2025-02-28_113713_401285_DPD__Mumps_verification_run_repeat_load.xlsx


 2024-04-03_105044_401285_DPD__H10ss_TECAN_POROS_Screen.xlsx



This is the file to be processed: 2024-04-03_105044_401285_DPD__H10ss_TECAN_POROS_Screen.xlsx

Please specify the final volume below, unit: uL


...

Please specify the final concentration below, unit: mg/ml


 0.1


Please specify the output folder name you would like to store the output files. 
Please do not include spaces in your folder name.


 out_put



This is the output folder your params files will be stored: out_put


# Read data in, clean and store data into dataframes grouped by Plate ID.

In [3]:
# Construct the file path
file_path = os.path.join(folder_name, raw_data)

# Read in the data
df = pd.read_excel(file_path, sheet_name='Report')
print(df.head())

              Report           Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0                NaN                  NaN        NaN        NaN        NaN   
1               Info                  NaN        NaN        NaN        NaN   
2               Date  03/04/2024 10:50:44        NaN        NaN        NaN   
3  Test performed by          DPD_Members        NaN        NaN        NaN   
4         Instrument               401285        NaN        NaN        NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0        NaN        NaN        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN        NaN        NaN         NaN  
3        NaN        NaN        NaN        NaN        NaN         NaN  
4        NaN        NaN        NaN        NaN        NaN         NaN  


In [5]:
# Find the row index where 'Plate ID' is located in the first column
plate_id_row = df[df.iloc[:, 0] == 'Plate ID'].index

# Check if 'Plate ID' exists
if not plate_id_row.empty:
    # Use the row as the new header
    new_header = df.iloc[plate_id_row[0]]
    df = df[plate_id_row[0] + 1:]  # Remove the header row from the DataFrame
    df.columns = new_header
    print(df.head())
else:
    print("'Plate ID' not found in the first column.")

23 Plate ID Plate\nPosition Sample name    Sample group Pump  \
24  Plate 1              A1   sample_A1  Sample Group 1    0   
25  Plate 1              B1   sample_B1  Sample Group 2    0   
26  Plate 1              C1   sample_C1  Sample Group 3    0   
27  Plate 1              D1   sample_D1  Sample Group 4    0   
28  Plate 1              E1   sample_E1  Sample Group 5    0   

23 Concentration\n(mg/ml) E1% Background (A280)  A280 A260/A280  \
24                   1.71  10              0.91  1.71      1.19   
25                   1.85  10              0.95  1.85      1.19   
26                   1.87  10              0.99  1.87      1.19   
27                   1.73  10              1.01  1.73      1.18   
28                    1.8  10              0.98   1.8      1.18   

23          Plate type  
24  High Lunatic Plate  
25  High Lunatic Plate  
26  High Lunatic Plate  
27  High Lunatic Plate  
28  High Lunatic Plate  


In [7]:
# find all the unique entries in "Plate ID"
unique_plate_ids = df['Plate ID'].unique()
unique_plate_ids # list the number of plates avaiable in the dataset

array(['Plate 1', 'Plate 2'], dtype=object)

In [9]:
# Create new dataframes by extracting column "Plate\nPosition", "Concentration\n(mg/ml)" and "E1%", refer to "Plate ID" to distinguish data into different dataframes
# Extract relevant columns and group by 'Plate ID'
relevant_cols = ['Plate\nPosition', 'Concentration\n(mg/ml)', 'E1%']
grouped = df.groupby('Plate ID')

# Create an empty list to store the plate dataframe names
plate_names = []

# Create new dataframes for each Plate ID
for plate_id, group in grouped:
    new_df = group[relevant_cols].copy()  # Create a copy to avoid SettingWithCopyWarning
    # Create a variable name for the new DataFrame based on the Plate ID
    # Replace any invalid characters in the Plate ID with underscores
    valid_plate_id = plate_id.replace(' ', '_').replace('\n', '_').replace('/', '_').replace('\\', '_')
    variable_name = f"{valid_plate_id}"
    globals()[variable_name] = new_df
    
    # Append the variable name to the list
    plate_names.append(variable_name)

# Run the code below and you will get the dataframes available by Plate ID

In [11]:
print(plate_names)  # Print the list of dataframe names

['Plate_1', 'Plate_2']


# Calculate how much sample and buffer were needed to make the concentration adjustment

## First, convert data to numeric data

In [13]:
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # Get a list of columns to convert (excluding "Plate\nPosition")
    cols_to_convert = [col for col in df.columns if col != 'Plate\nPosition']

    # Convert the selected columns to numeric
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

    print(f"DataFrame: {plate_name}")
    print(df.dtypes)  # Print the data types of each column
    print("-" * 20)  # Print a separator line

DataFrame: Plate_1
23
Plate\nPosition            object
Concentration\n(mg/ml)    float64
E1%                         int64
dtype: object
--------------------
DataFrame: Plate_2
23
Plate\nPosition            object
Concentration\n(mg/ml)    float64
E1%                         int64
dtype: object
--------------------


## Clean the data

In [15]:
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # replace all N/A value by 0
    df = df.infer_objects(copy=False) #convert object-type columns to more appropriate types
    df.fillna(0, inplace=True)

## Calculate sample and buffer volume and store to new columns
1. If starting concentration is below final concentration, no dilution occurs and final concentration will be the same as starting concentration.
2. If sample concentration is 0 then no sample or buffer will be added to the corresponding well. 

In [17]:
# create new columns in the dataframe: "Final Concentration (mg/ml)", "Final Volume (uL)", "Sample Volume (ul)" and "Buffer Volume (uL)
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # Create 'Final Concentration (mg/ml)'
    df['Final Concentration (mg/ml)'] = df['Concentration\n(mg/ml)'].apply(
        lambda x: 0 if x == 0 else (final_concentration if x > final_concentration else x)
    )
    
    # Create 'Final Volume (uL)'
    df['Final Volume (uL)'] = df.apply(
        lambda row: 0 if row['Concentration\n(mg/ml)'] == 0 else final_volume,
        axis=1
    )

    # Create 'Sample Volume (ul)' 
    df['Sample Volume (ul)'] = df.apply(
        lambda row: 0 if row['Concentration\n(mg/ml)'] == 0 
        else (final_volume if row['Concentration\n(mg/ml)'] <= final_concentration
        else (final_concentration * final_volume / row['Concentration\n(mg/ml)'])),
        axis=1
    )

    # Create 'Buffer Volume (ul)'
    df['Buffer Volume (ul)'] = df['Final Volume (uL)'] - df['Sample Volume (ul)']  # Use 'Final Volume (uL)'

## Covert volume numbers to integers: easier to pipet on TECAN

In [19]:
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # Get a list of columns to convert (excluding "Plate\nPosition")
    cols_to_convert = ['Sample Volume (ul)', 'Buffer Volume (ul)']

    # Convert the selected columns to numeric
    df[cols_to_convert] = df[cols_to_convert].astype(int)

    # save the processed data to a .csv file
    csv_filename = os.path.join(output_folder, plate_name + ".csv")
    df.to_csv(csv_filename, index=False)

display(Plate_1)

23,Plate\nPosition,Concentration\n(mg/ml),E1%,Final Concentration (mg/ml),Final Volume (uL),Sample Volume (ul),Buffer Volume (ul)
24,A1,1.71,10,0.10,200,11,188
25,B1,1.85,10,0.10,200,10,189
26,C1,1.87,10,0.10,200,10,189
27,D1,1.73,10,0.10,200,11,188
28,E1,1.80,10,0.10,200,11,188
...,...,...,...,...,...,...,...
115,D12,0.02,10,0.02,200,200,0
116,E12,0.04,10,0.04,200,200,0
117,F12,0.03,10,0.03,200,200,0
118,G12,0.00,10,0.00,0,0,0


# A quick visualization to check the data in a 96-well format

## Define functions to create 96well plate layout to store results

In [21]:
def create_96_well_plate():
    """Creates a dictionary representing a 96-well plate."""
    rows = "ABCDEFGH"
    cols = range(1, 13)
    plate = {}
    for row in rows:
        for col in cols:
            well_name = f"{row}{col:d}"
            plate[well_name] = None  # Initialize each well as empty
    return plate

def print_plate(plate):
    """Prints the 96-well plate in a readable format."""
    rows = "ABCDEFGH"
    cols = range(1, 13)
    print("   ", end="")
    for col in cols:
        print(f"{col:d},", end="")
    print()
    for row in rows:
        print(f"{row}  ", end="")
        for col in cols:
            well_name = f"{row}{col:d}"
            print(f"{plate[well_name] if plate[well_name] is not None else '0'},", end="") 
        print()

# Example usage:
plate_96 = create_96_well_plate()
print("Initial 96-well plate:")
print_plate(plate_96)

Initial 96-well plate:
   1,2,3,4,5,6,7,8,9,10,11,12,
A  0,0,0,0,0,0,0,0,0,0,0,0,
B  0,0,0,0,0,0,0,0,0,0,0,0,
C  0,0,0,0,0,0,0,0,0,0,0,0,
D  0,0,0,0,0,0,0,0,0,0,0,0,
E  0,0,0,0,0,0,0,0,0,0,0,0,
F  0,0,0,0,0,0,0,0,0,0,0,0,
G  0,0,0,0,0,0,0,0,0,0,0,0,
H  0,0,0,0,0,0,0,0,0,0,0,0,


## Assign sample volume and buffer volume data to 96-well plates and print the layouts

In [23]:
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # Assign sample volumes from dataframe to plate_96
    for index, row in df.iterrows():
        well_position = row["Plate\nPosition"] 
        sample_volume = row["Sample Volume (ul)"]  
        plate_96[well_position] = sample_volume

    print("\n" + f"{plate_name}" + " with sample volumes:")
    print_plate(plate_96)

    # Assign buffer volumes from dataframe to plate_96
    for index, row in df.iterrows():
        well_position = row["Plate\nPosition"] 
        buffer_volume = row["Buffer Volume (ul)"]  
        plate_96[well_position] = buffer_volume

    print("\n" + f"{plate_name}" + " with buffer volumes:")
    print_plate(plate_96)


Plate_1 with sample volumes:
   1,2,3,4,5,6,7,8,9,10,11,12,
A  11,200,200,200,200,15,3,5,30,166,200,200,
B  10,200,200,200,200,200,19,3,4,15,76,200,
C  10,200,200,200,200,200,15,4,4,14,74,200,
D  11,0,200,200,200,200,38,0,4,15,86,200,
E  11,200,200,200,200,200,111,6,3,9,37,200,
F  11,200,200,200,200,58,12,4,5,18,142,200,
G  11,200,200,200,200,8,3,8,42,200,200,0,
H  10,200,133,8,2,7,125,200,200,200,200,0,

Plate_1 with buffer volumes:
   1,2,3,4,5,6,7,8,9,10,11,12,
A  188,0,0,0,0,184,196,194,169,33,0,0,
B  189,0,0,0,0,0,180,196,195,184,123,0,
C  189,0,0,0,0,0,184,195,195,185,125,0,
D  188,0,0,0,0,0,161,0,195,184,113,0,
E  188,0,0,0,0,0,88,193,196,190,162,0,
F  188,0,0,0,0,141,187,195,194,181,57,0,
G  188,0,0,0,0,191,196,191,157,0,0,0,
H  190,0,66,191,197,192,75,0,0,0,0,0,

Plate_2 with sample volumes:
   1,2,3,4,5,6,7,8,9,10,11,12,
A  11,200,200,200,200,15,3,5,29,153,200,200,
B  10,0,200,200,200,200,18,3,4,16,74,200,
C  10,200,200,200,200,200,15,4,4,14,71,200,
D  11,0,200,200,200,200,3

# Transfer data from plate dataframe to 96well plate and generate params files

## Define a function to create params.txt file

In [25]:
def create_params_file(df, filename):
    """
    Creates a parameters file, filling all missing wells with 0,
    even if the well position is absent in the source dataframe.
    """

    with open(filename, "w") as f:
        f.write("# Tip Mask [TIP_MASK]\n")
        f.write("255\n")
        f.write("# Num Rows [iNumRows]\n")
        f.write("8\n")
        f.write("# Num Columns [iNumColumns]\n")
        f.write("12\n")

        f.write("# Sample Volumes\n")
        for row in "ABCDEFGH":
            sample_volumes = []
            for col in range(1, 13):
                well_position = f"{row}{col}"
                # Check if well_position exists in the DataFrame
                if well_position in df["Plate\nPosition"].values:
                    sample_volume = df.loc[df["Plate\nPosition"] == well_position, "Sample Volume (ul)"].iloc[0]
                    sample_volumes.append(str(sample_volume))
                else:
                    # If well_position is not found, append 0
                    sample_volumes.append("0")
            f.write(",".join(sample_volumes) + "\n")

        # Similar change for Buffer Volumes
        f.write("# Buffer Volumes\n")
        for row in "ABCDEFGH":
            buffer_volumes = []
            for col in range(1, 13):
                well_position = f"{row}{col}"
                if well_position in df["Plate\nPosition"].values:
                    buffer_volume = df.loc[df["Plate\nPosition"] == well_position, "Buffer Volume (ul)"].iloc[0]
                    buffer_volumes.append(str(buffer_volume))
                else:
                    buffer_volumes.append("0")
            f.write(",".join(buffer_volumes) + "\n")

# Loop through dataframes and create files
for plate_name in plate_names:
    df = globals()[plate_name]  # Get the DataFrame using its name

    # Construct the full file path
    filename = os.path.join(output_folder, plate_name + "_params.txt")  
    
    create_params_file(df, filename)